In [5]:
import tokenizer as tok_tests
import collections

In [19]:
def corpus_common_tokens(strings):
    dict = collections.defaultdict(lambda: 0)
    for s in strings:
        for st in s.split():
            dict[st] += 1

In [ ]:
c = collections.Counter()

In [20]:
import torchtext

In [21]:
torchtext.__version__

'0.6.0'